# 07 — Power BI Visualização / Visualization

Este notebook demonstra um fluxo completo para integração entre Python e Power BI Desktop/Service, com exportação/importação de dados, criação de medidas/KPIs e visuais avançados.

This notebook shows an end-to-end workflow to integrate Python with Power BI Desktop/Service, including data export/import, KPI measures, and advanced visuals.

## Objetivos / Objectives
- Exportar dados tratados do Python para CSV/Parquet a serem importados no Power BI.
- Demonstrar scripts Python como fonte no Power BI e uso de pandas.
- Criar KPIs (taxa de conversão, ticket médio, churn) e cartões/indicadores.
- Explorar visuais avançados (Decomposition Tree, Key Influencers, Forecast).
- Construir interações (slicers, drill-down) e navegação.

- Export cleaned data from Python to CSV/Parquet for Power BI import.
- Show Python scripts as a data source in Power BI using pandas.
- Build KPIs (conversion rate, average order value, churn) and cards/indicators.
- Explore advanced visuals (Decomposition Tree, Key Influencers, Forecast).
- Build interactions (slicers, drill-down) and navigation.

In [ ]:
# %% [markdown]
# Instalação e imports / Install and imports
# Execute se necessário / Run if needed
%pip -q install pandas pyarrow plotly prophet scikit-learn

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from datetime import datetime, timedelta
import plotly.express as px


## Dados de exemplo / Sample data
Vamos simular um dataset de pedidos com clientes, datas, valores e conversões.

We will simulate an orders dataset with customers, dates, amounts, and conversions.

In [ ]:
rng = pd.date_range('2024-01-01', periods=270, freq='D')
np.random.seed(42)
n_customers = 500
customers = [f'C{{i:04d}}' for i in range(1, n_customers+1)]
df = pd.DataFrame({
    'date': np.random.choice(rng, size=5000),
    'customer_id': np.random.choice(customers, size=5000),
    'channel': np.random.choice(['Web','App','Store'], size=5000, p=[0.5,0.3,0.2]),
    'segment': np.random.choice(['SMB','Enterprise','Consumer'], size=5000, p=[0.4,0.2,0.4]),
    'amount': np.round(np.random.gamma(2.0, 50.0, size=5000), 2)
})
df['converted'] = (np.random.rand(len(df)) < 0.35).astype(int)
df['orders'] = np.where(df['converted']==1, np.random.poisson(1.2, len(df)), 0)
df.head()


## Exportação de dados / Data export
Salve dados em formatos ideais para o Power BI (CSV/Parquet).

Save data in Power BI-friendly formats (CSV/Parquet).

In [ ]:
export_dir = 'data_exports'
import os
os.makedirs(export_dir, exist_ok=True)
csv_path = os.path.join(export_dir, 'orders_sample.csv')
parquet_path = os.path.join(export_dir, 'orders_sample.parquet')
df.to_csv(csv_path, index=False, encoding='utf-8')
df.to_parquet(parquet_path, index=False)
csv_path, parquet_path


### Importar no Power BI / Import into Power BI
1) Power BI Desktop → Obter dados → Texto/CSV ou Parquet → aponte para os arquivos exportados.
2) Alternativamente, use 'Script do Python' como fonte e copie o snippet abaixo.

1) Power BI Desktop → Get Data → Text/CSV or Parquet → point to exported files.
2) Alternatively, use 'Python script' as source and paste the snippet below.

In [ ]:
# Snippet para usar no conector de Script do Python do Power BI /
# Snippet to use in Power BI Python script connector
import pandas as pd
df = pd.read_csv(r'FULL_PATH_TO/data_exports/orders_sample.csv')
# Opcional: transformações rápidas / Optional: quick transforms
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M').dt.to_timestamp()
dataset = df  # o Power BI espera um dataframe chamado 'dataset' / Power BI expects 'dataset'


## KPIs e Métricas / KPIs and Metrics
Definições de KPIs usados no relatório.

Definitions of KPIs used in the report.

In [ ]:
# KPIs calculados no Python (poderão virar medidas DAX no Power BI)
agg = df.groupby('customer_id').agg({
    'converted':'max',
    'amount':'sum',
    'orders':'sum'
}).rename(columns={'converted':'has_converted'})
agg['aov'] = agg['amount'] / agg['orders'].replace(0, np.nan)
kpis = {
    'conversion_rate': agg['has_converted'].mean(),
    'avg_order_value': agg['aov'].mean(skipna=True),
    'arpu': agg['amount'].mean(),
    'customers': agg.shape[0]
}
kpis


### DAX de exemplo / Sample DAX
Copie para medidas no Power BI. / Copy into measures in Power BI.

- Conversion Rate = DIVIDE(CALCULATE(DISTINCTCOUNT(Orders[customer_id]), Orders[converted] = 1), DISTINCTCOUNT(Orders[customer_id]))
- AOV = DIVIDE(SUM(Orders[amount]), SUM(Orders[orders]))
- Churn (simplificado) = 1 - DIVIDE(DISTINCTCOUNT(Active[customer_id]), DISTINCTCOUNT(Customers[customer_id]))

## Visuais avançados / Advanced visuals
Passos recomendados no Power BI Desktop: / Recommended steps in Power BI Desktop:
1) Key Influencers: analisar fatores que impactam Converted.
2) Decomposition Tree: decompor Amount por Canal/Segmento/Data.
3) Forecast (linha temporal): prever Amount por mês.
4) Botões de navegação e bookmarks para histórias.

1) Key Influencers: analyze drivers of Converted.
2) Decomposition Tree: decompose Amount by Channel/Segment/Date.
3) Time series Forecast: forecast Amount by month.
4) Navigation buttons and bookmarks for storytelling.

## Gráficos interativos em Python / Interactive charts in Python
Úteis para validar hipóteses antes de publicar no Power BI.

Useful to validate hypotheses before publishing to Power BI.

In [ ]:
df_month = (df.assign(month=lambda d: pd.to_datetime(d['date']).dt.to_period('M').dt.to_timestamp())
              .groupby(['month','channel'], as_index=False)['amount'].sum())
fig = px.line(df_month, x='month', y='amount', color='channel', title='Monthly Amount by Channel')
fig.show()


## Integração Python no Power BI / Python integration in Power BI
- Ative scripts Python em Arquivo → Opções → Scripts do Python.
- Configure a pasta onde estão suas dependências/venv.
- Use visuals de Python para gráficos customizados quando necessário.

- Enable Python scripting in File → Options → Python scripting.
- Point to your Python home/venv.
- Use Python visuals for custom charts when needed.

## Publicação / Publishing
- Publique do Desktop para o Power BI Service.
- Configure atualização agendada se a origem estiver no OneDrive/SharePoint.
- Garanta gateways para fontes on-premises.

- Publish from Desktop to Power BI Service.
- Configure scheduled refresh if source is on OneDrive/SharePoint.
- Ensure gateways for on-premises sources.

## Checklist didático / Teaching checklist
- [ ] Exportou CSV/Parquet do Python?
- [ ] Importou no Power BI ou usou Script do Python?
- [ ] Criou medidas DAX para KPIs?
- [ ] Montou visuais avançados (Key Influencers, Decomposition, Forecast)?
- [ ] Ajustou interações (slicers, drill)?
- [ ] Publicou e configurou atualização?

- [ ] Exported CSV/Parquet from Python?
- [ ] Imported into Power BI or used Python script?
- [ ] Created DAX measures for KPIs?
- [ ] Built advanced visuals (Key Influencers, Decomposition, Forecast)?
- [ ] Tuned interactions (slicers, drill)?
- [ ] Published and scheduled refresh?